In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score, recall_score, precision_score
from sklearn.metrics import f1_score, make_scorer
from sklearn.model_selection import cross_validate, StratifiedShuffleSplit

from xgboost import XGBClassifier

In [2]:
from sklearn.model_selection import GridSearchCV

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
X = np.load('drive/MyDrive/DATA/data_2/X.npy').astype(np.int8)
y = np.load('drive/MyDrive/DATA/data_2/y.npy').astype(np.int8)

In [5]:
scoring = {'accuracy': 'accuracy',
           'F1-score': make_scorer(f1_score, average='micro'),
           }

In [16]:
parameters = {'min_child_weight': [1, 5, 10, 15],
              'scale_pos_weight': [0.2, 0.25, 0.3],
              'max_depth': [1, 2, 3],
              'subsample': [0.8, 1.0],
              }
clf = XGBClassifier(n_estimators=50,
                    predictor='gpu_predictor',
                    use_label_encoder=False)
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

gs = GridSearchCV(clf, parameters, scoring, cv=split, verbose=4, refit='F1-score', n_jobs=2)
gs.fit(X, y)

Fitting 1 folds for each of 72 candidates, totalling 72 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  21 tasks      | elapsed:  7.1min
[Parallel(n_jobs=2)]: Done  72 out of  72 | elapsed: 44.3min finished


GridSearchCV(cv=StratifiedShuffleSplit(n_splits=1, random_state=42, test_size=0.2,
            train_size=None),
             error_score=nan,
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=1, gamma=0,
                                     learning_rate=0.1, max_delta_step=0,
                                     max_depth=3, min_child_weight=1,
                                     missing=None, n_estimators=50, n_jobs=1,
                                     nthread=Non...
                                     subsample=1, use_label_encoder=False,
                                     verbosity=1),
             iid='deprecated', n_jobs=2,
             param_grid={'max_depth': [1, 2, 3],
                         'min_child_weight': [1, 5, 10, 15],
                         'scale_pos_weight': [0.2, 0.25, 0.3],
                         'su

In [19]:
y = np.where(y == 1, 0, 1)
np.unique(y)

array([0, 1])

In [22]:
scoring = {'accuracy': 'accuracy',
           'precision': 'precision',
           'recall': 'recall',
           'F1-score': 'f1',
           'recall -': make_scorer(recall_score, pos_label=0),
           'precision -': make_scorer(precision_score, pos_label=0, zero_division=0),
           }


In [28]:
gs.best_estimator_.n_estimators = 100

In [29]:
gs.best_estimator_

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=10, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic',
              predictor='gpu_predictor', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=0.3, seed=None, silent=None,
              subsample=1.0, use_label_encoder=False, verbosity=1)

In [30]:
clf = gs.best_estimator_
scores = cross_validate(clf, X, y, cv=5, scoring=scoring, n_jobs=-1)
mean_scores = pd.DataFrame(scores).apply(np.mean)

In [31]:
scores_df = pd.DataFrame(columns=list(scoring.keys()))


scores_df.loc[type(clf).__name__] = mean_scores.values[2:]
scores_df

,accuracy,precision,recall,F1-score,recall -,precision -
XGBClassifier,0.704423,0.761594,0.883026,0.817807,0.163934,0.322101
